# Test with baseline model

In [1]:
!pip install dataset accelerate databench_eval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency res

In [2]:
import pandas as pd
import subprocess
import shlex
import zipfile
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from databench_eval import Runner, Evaluator, utils

### set up

In [3]:
tokenizer = AutoTokenizer.from_pretrained("stabilityai/stable-code-3b")
model = AutoModelForCausalLM.from_pretrained(
  "stabilityai/stable-code-3b",
  torch_dtype="auto",
)
model.cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/4.76k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/602 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/610M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

StableLmForCausalLM(
  (model): StableLmModel(
    (embed_tokens): Embedding(50304, 2560)
    (layers): ModuleList(
      (0-31): 32 x StableLmDecoderLayer(
        (self_attn): StableLmSdpaAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (o_proj): Linear(in_features=2560, out_features=2560, bias=False)
          (attention_dropout): Dropout(p=0.0, inplace=False)
          (rotary_emb): StableLmRotaryEmbedding()
        )
        (mlp): StableLmMLP(
          (gate_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (up_proj): Linear(in_features=2560, out_features=6912, bias=False)
          (down_proj): Linear(in_features=6912, out_features=2560, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affin

In [4]:
def call_model(prompts):
  """
  tokenize prompt, model generate;
  """
  results = []
  for p in prompts:
      inputs = tokenizer(p, return_tensors="pt").to(model.device)
      tokens = model.generate(
          **inputs,
          max_new_tokens=128,
          temperature=0.2,
          do_sample=True,
          pad_token_id=tokenizer.eos_token_id
      )
      result = tokenizer.decode(tokens[0], skip_special_tokens=True)
      results.append(result)
  return results

In [5]:
# load data and evaluator
qa = utils.load_qa(name="semeval", split="dev")
evaluator = Evaluator(qa=qa)

README.md:   0%|          | 0.00/43.6k [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/65 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/49 [00:00<?, ?it/s]

qa.parquet:   0%|          | 0.00/6.74k [00:00<?, ?B/s]

qa.parquet:   0%|          | 0.00/7.29k [00:00<?, ?B/s]

...

qa.parquet:   0%|          | 0.00/6.92k [00:00<?, ?B/s]

qa.parquet:   0%|          | 0.00/7.18k [00:00<?, ?B/s]

qa.parquet:   0%|          | 0.00/7.20k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/988 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/320 [00:00<?, ? examples/s]

In [9]:
qa[0]

{'question': 'Is the most favorited author mainly communicating in Spanish?',
 'answer': 'True',
 'type': 'boolean',
 'columns_used': "['favorites', 'lang']",
 'column_types': "['category', 'category']",
 'sample_answer': 'True',
 'dataset': '050_ING'}

### overridden compare in Evaluator

In [10]:
# reason: qa[211] always throw error message:
# Semantic not supported:  category
# qa[211]['type']= ' category', the string start with space;
# With the default evaluation see code line 21, elif semantic == "category"
# this line result the evaluator always throw Exception(f"Semantic not supported: {semantic}")

In [6]:
err_dict,none_str_dict = {},{}
for i in range(len(qa)):
   for value_ in qa[i].values():
    if isinstance(value_, str):
      if value_.startswith(" "):
        err_dict[i]=qa[i]
    else:
      none_str_dict[i]=qa[i]
err_dict # see index 211

{191: {'question': 'In which province are the most cars located?',
  'answer': ' Madrid',
  'type': 'category',
  'columns_used': "['province']",
  'column_types': "['category']",
  'sample_answer': 'Madrid',
  'dataset': '059_Second'},
 209: {'question': "On weekdays, what's the most commonly bought item?",
  'answer': 'Coffee',
  'type': 'category',
  'columns_used': '[Item, weekday_weekend]',
  'column_types': "['category', 'category']",
  'sample_answer': ' Coffee',
  'dataset': '060_Bakery'},
 211: {'question': 'During which period of the day is Brownie" most frequently bought?"',
  'answer': ' afternoon',
  'type': ' category',
  'columns_used': '[Item, period_day]',
  'column_types': "['category', 'category']",
  'sample_answer': ' afternoon',
  'dataset': '060_Bakery'}}

In [7]:
def custom_compare(value, truth, semantic):
  # override the default compare method;
    semantic = semantic.strip()  # add this line to remove the space;

    if semantic == "boolean":
        return str(value) == str(truth)
    elif semantic == "category":
        return str(value) == str(truth)
    elif semantic == "number":
        try:
            value_cleaned = ''.join(char for char in str(value) if char.isdigit() or char in ['.', '-'])
            truth_cleaned = ''.join(char for char in str(truth) if char.isdigit() or char in ['.', '-'])
            return round(float(value_cleaned), 2) == round(float(truth_cleaned), 2)
        except :
            return False
    elif semantic == "list[category]":
        try:
            value_list = [item.strip() for item in str(value).strip('[]').split(',')]
            truth_list = [item.strip() for item in str(truth).strip('[]').split(',')]
            if len(value_list) != len(truth_list):
                return False
            return set(value_list) == set(truth_list)
        except Exception as exc:
            return False

    elif semantic == "list[number]":
        try:
            value_list = sorted(round(float(''.join(c for c in v.strip() if c.isdigit() or c in ['.', '-'])), 2) for v in str(value).strip('[]').split(',') if v.strip())
            truth_list = sorted(round(float(''.join(c for c in t.strip() if c.isdigit() or c in ['.', '-'])), 2) for t in str(truth).strip('[]').split(',') if t.strip())
            if len(value_list) != len(truth_list):
                return False
            return set(value_list) == set(truth_list)
        except Exception as exc:
            return False

    else:
        raise ValueError(f"Semantic not supported: '{semantic}'")

In [8]:
custom_evaluator = Evaluator(compare=custom_compare,qa=qa)

### response

In [16]:
prompts = [] # prompt
code_res_full = [] # generated code, full text
code_res_run = [] # generated code, trunct till the first \n, what is actually running

In [17]:
# prompt
def example_generator(row: dict) -> str:
    """IMPORTANT:
    **Only the question and dataset keys will be available during the actual competition**.
    You can, however, try to predict the answer type or columns used
    with another modeling task if that helps, then use them here.
    """
    dataset = row["dataset"]
    question = row["question"]
    df = utils.load_table(dataset)
    prompt =  f"""
# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -> int:
    df.columns=["A"]
    return df.shape[0]

# TODO: complete the following function in one line. It should give the answer to: {question}
def answer(df: pd.DataFrame) -> {row["type"]}:
    df.columns = {list(df.columns)}
    return"""

    prompts.append(prompt) # append to promps
    return prompt




# code -> the actual data
def example_postprocess(response: str, dataset: str, loader):
    try:
        resp_full = response.split("return")[2]
        resp_run = response.split("return")[2].split("\n")[0].strip().replace("[end of text]", "")

        code_res_full.append(resp_full) # append to code_responses
        code_res_run.append(resp_run)

        df = loader(dataset)
        lead = """
def answer(df):
    return """
        exec(
            "global ans\n"
            + lead
            + response.split("return")[2]
            .split("\n")[0]
            .strip()
            .replace("[end of text]", "")
            + f"\nans = answer(df)"
        )

        # no true result is > 1 line atm, needs 1 line for txt format
        return ans.split("\n")[0] if "\n" in str(ans) else ans
    except Exception as e:
        return f"__CODE_ERROR__: {e}"

In [18]:
runner = Runner(
    model_call=call_model,
    prompt_generator=example_generator,
    postprocess=lambda response, dataset: example_postprocess(
        response, dataset, utils.load_table
    ),
    qa=qa,
    batch_size=10,
)

In [21]:
responses = runner.run()

 12%|█▎        | 4/32 [04:33<31:17, 67.04s/it]<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
 19%|█▉        | 6/32 [06:49<29:18, 67.63s/it]<string>:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain curr

In [22]:
#print(f"DataBench accuracy is {evaluator.eval(responses)}")  # ~0.15
custom_evaluator.eval(responses)

100%|██████████| 320/320 [00:00<00:00, 51705.73it/s]


0.446875

In [23]:
df_responses = {
    'prompt': prompts,
    'full_code_response': code_res_full,
    "code_implement":code_res_run,
    'data_ans': responses,
    "gold": qa["answer"]
}

df = pd.DataFrame(df_responses)
df

,prompt,full_code_response,code_implement,data_ans,gold
0,\n# TODO: complete the following function in o...,"df.loc[df['lang'] == 'es', 'favorites'].sum()...","df.loc[df['lang'] == 'es', 'favorites'].sum() ...",True,True
1,\n# TODO: complete the following function in o...,df.author_name.value_counts().idxmax() == 'or...,df.author_name.value_counts().idxmax() == 'ori...,False,True
2,\n# TODO: complete the following function in o...,"df.loc[df['retweets'] == 0, 'author_id'].any(...","df.loc[df['retweets'] == 0, 'author_id'].any()",True,True
3,\n# TODO: complete the following function in o...,df.links.isnull().any()\n\n# TODO: complete t...,df.links.isnull().any(),False,True
4,\n# TODO: complete the following function in o...,df.author_id.nunique()\n\n# TODO: complete th...,df.author_id.nunique(),3943,3765
...,...,...,...,...,...
315,\n# TODO: complete the following function in o...,df.groupby('Country').sum()['Quantity'].sort_...,df.groupby('Country').sum()['Quantity'].sort_v...,__CODE_ERROR__: category type does not support...,"['Saudi Arabia', 'Bahrain']"
316,\n# TODO: complete the following function in o...,"df.nlargest(5, 'Quantity')['InvoiceNo'].tolis...","df.nlargest(5, 'Quantity')['InvoiceNo'].tolist()","[581483.0, 541431.0, 578841.0, 542504.0, 57300...","['581483', '541431', '578841', '542504', '5730..."
317,\n# TODO: complete the following function in o...,"df.sort_values('Quantity', ascending=False).h...","df.sort_values('Quantity', ascending=False).he...","[23843, 23166, 84826, 37413]","[23843, 21366, 23005, 23005, 84347]"
318,\n# TODO: complete the following function in o...,df.groupby('Description').sum().sort_values('...,df.groupby('Description').sum().sort_values('Q...,__CODE_ERROR__: category type does not support...,"[1, 2, 12, 6]"


In [26]:
from google.colab import drive
drive.mount('/content/drive')
df.to_csv('/content/drive/My Drive/task_8/response_stable-code-3b.csv', index=False)

Mounted at /content/drive


### lite -> to create a lite df

In [9]:
prompts_lite = [] # prompt
code_res_full_lite = [] # generated code, full text
code_res_run_lite = [] # generated code which is actually running

In [10]:
# prompt
def example_generator(row: dict) -> str:
    """IMPORTANT:
    **Only the question and dataset keys will be available during the actual competition**.
    You can, however, try to predict the answer type or columns used
    with another modeling task if that helps, then use them here.
    """
    dataset = row["dataset"]
    question = row["question"]
    df = utils.load_table(dataset)
    prompt =  f"""
# TODO: complete the following function in one line. It should give the answer to: How many rows are there in this dataframe?
def example(df: pd.DataFrame) -> int:
    df.columns=["A"]
    return df.shape[0]

# TODO: complete the following function in one line. It should give the answer to: {question}
def answer(df: pd.DataFrame) -> {row["type"]}:
    df.columns = {list(df.columns)}
    return"""

    prompts_lite.append(prompt) # append to promps
    return prompt




# code -> the actual data
def example_postprocess(response: str, dataset: str, loader):
    try:
        resp_full_lite = response.split("return")[2]
        resp_run_lite = response.split("return")[2].split("\n")[0].strip().replace("[end of text]", "")

        code_res_full_lite.append(resp_full_lite)
        code_res_run_lite.append(resp_run_lite)

        df = loader(dataset)
        lead = """
def answer(df):
    return """
        exec(
            "global ans\n"
            + lead
            + response.split("return")[2]
            .split("\n")[0]
            .strip()
            .replace("[end of text]", "")
            + f"\nans = answer(df)"
        )

        # no true result is > 1 line atm, needs 1 line for txt format
        return ans.split("\n")[0] if "\n" in str(ans) else ans
    except Exception as e:
        return f"__CODE_ERROR__: {e}"

In [11]:
runner_lite = Runner(
    model_call=call_model,
    prompt_generator=example_generator,
    postprocess=lambda response, dataset: example_postprocess(
        response, dataset, utils.load_sample
    ),
    qa=qa,
    batch_size=10,
)

In [12]:
responses_lite = runner_lite.run()

100%|██████████| 32/32 [44:15<00:00, 82.97s/it]


In [13]:
print(
    f"DataBench_lite accuracy is {custom_evaluator.eval(responses_lite, lite=True)}"
) # ~0.07

100%|██████████| 320/320 [00:00<00:00, 38256.11it/s]

DataBench_lite accuracy is 0.4375


In [20]:
df_responses_lite = {
    'prompt': prompts_lite,
    'full_code_response': code_res_full_lite,
    "running_code_response":code_res_run_lite,
    'data_ans': responses_lite,
    "gold": qa["answer"]
}

df_lite = pd.DataFrame(df_responses_lite)
df_lite

,prompt,full_code_response,running_code_response,data_ans,gold
0,\n# TODO: complete the following function in o...,"df.loc[df['lang'] == 'es', 'favorites'].sum()...","df.loc[df['lang'] == 'es', 'favorites'].sum() ...",True,True
1,\n# TODO: complete the following function in o...,df.author_name.value_counts().idxmax() == 'or...,df.author_name.value_counts().idxmax() == 'ori...,False,True
2,\n# TODO: complete the following function in o...,df.loc[df['retweets'] == 0].empty\n\n# TODO: ...,df.loc[df['retweets'] == 0].empty,False,True
3,\n# TODO: complete the following function in o...,df.loc[df['links'].isna()]\n\n# TODO: complet...,df.loc[df['links'].isna()],__CODE_ERROR__: 'DataFrame' object has no attr...,True
4,\n# TODO: complete the following function in o...,df.author_id.nunique()\n\n# TODO: complete th...,df.author_id.nunique(),__CODE_ERROR__: 'DataFrame' object has no attr...,3765
...,...,...,...,...,...
315,\n# TODO: complete the following function in o...,df.groupby('Country').sum()['Quantity'].sort_...,df.groupby('Country').sum()['Quantity'].sort_v...,"[Australia, United Kingdom]","['Saudi Arabia', 'Bahrain']"
316,\n# TODO: complete the following function in o...,"df.nlargest(5, 'Quantity')['InvoiceNo'].tolis...","df.nlargest(5, 'Quantity')['InvoiceNo'].tolist()","[548005, 555200, 573399, 577076, 568909]","['581483', '541431', '578841', '542504', '5730..."
317,\n# TODO: complete the following function in o...,"df.nlargest(4, 'Quantity')['StockCode'].tolis...","df.nlargest(4, 'Quantity')['StockCode'].tolist()","[21499, 71459, 21108, 22614]","[23843, 21366, 23005, 23005, 84347]"
318,\n# TODO: complete the following function in o...,df.Quantity.value_counts().head(4).index.toli...,df.Quantity.value_counts().head(4).index.tolist(),"[1, 2, 4, 12]","[1, 2, 12, 6]"


In [21]:
from google.colab import drive
drive.mount('/content/drive')
df_lite.to_csv('/content/drive/My Drive/task_8/response_lite_stable-code-3b.csv', index=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
